## Cifra repartidora
 Elecciones legislativas de Colombia de 2018 se realizaron el domingo 11 de marzo. En ellas se eligieron los miembros de ambas cámaras del Congreso de Colombia


In [ ]:
import colorama

In [1]:
# Funciones
def repartidora(votos: list, curules: int) -> int:
    values = []
    tabla = [[0 for col in range(curules)] for row in range(len(votos))]
    lista = -1
    for votos_lista in votos:
        lista += 1
        for item in range(curules):
            value = votos_lista/(item+1)
            values.append(value)
            tabla[lista][item] = value
    return round(sorted(values)[-curules-2])


def credenciales(votos: list, repartidora: int, umbral: int) -> int:
    curules = []
    for item in votos:
        if item >= umbral:
            curules.append(int(item/repartidora))
        else:
            curules.append(0)
    return curules


In [2]:
# Datos votación escrutinio final 2018
votos_listas = [2_508_534, 2_152_793, 1_930_197, 1_896_283, 1_854_349, 1_312_291,
                725_244, 520_462, 495_106, 464_916, 345_342, 103_368, 58_023, 55_587, 34_555, 13_439]
votos_blancos = 842_286
curules_senado_ordinaria = 100
curules_senado_especial = 8


In [3]:
# Cálculo de credenciales senado escrutinio final 2018
votos_validos = sum(votos_listas) + votos_blancos
umbral = votos_validos*(3/100)
cifra_repartidora = repartidora(votos_listas, curules_senado_ordinaria)
curules_por_lista = credenciales(votos_listas, cifra_repartidora, umbral)


In [4]:
print(f"*"*30)
print(f"Validos      : {votos_validos:15,.0f}")
print(f"Umbral       : {umbral:15,.0f}")
print(f"Repartidora  : {cifra_repartidora:15,.0f}")
print(f"*"*30)
for item in range(len(votos_listas)):
    print(f"|{item+1:3.0f} |{votos_listas[item]:15,.0f} |{curules_por_lista[item]:5.0f} |")
print(f"*"*30)
print(f"|  T |{sum(votos_listas):15,.0f} |{sum(curules_por_lista):5.0f} |")

******************************
Validos      :      15,312,775
Umbral       :         459,383
Repartidora  :         132,028
******************************
|  1 |      2,508,534 |   19 |
|  2 |      2,152,793 |   16 |
|  3 |      1,930,197 |   14 |
|  4 |      1,896,283 |   14 |
|  5 |      1,854,349 |   14 |
|  6 |      1,312,291 |    9 |
|  7 |        725,244 |    5 |
|  8 |        520,462 |    3 |
|  9 |        495,106 |    3 |
| 10 |        464,916 |    3 |
| 11 |        345,342 |    0 |
| 12 |        103,368 |    0 |
| 13 |         58,023 |    0 |
| 14 |         55,587 |    0 |
| 15 |         34,555 |    0 |
| 16 |         13,439 |    0 |
******************************
|  T |     14,470,489 |  100 |
